In [3]:
import sys, os
lib_path = os.path.abspath('../libs')
sys.path.append(lib_path)

Model Optimization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import math

from InputOptimization import InputOptimization

In [5]:
df = pd.read_csv('../00_Data/05_Encoded.csv')
df.head()

,Gender,Generation,Term Sub Reason,Employee Grade,Location,Career Bucket,Age Bucket,Manager ID,Talent,Department,active_months
0,1,2,1,1,1,1,2,102292,5,2,59.0
1,1,2,1,2,1,2,2,100411,5,3,4.0
2,0,2,1,1,1,1,2,102304,5,5,26.0
3,0,3,1,3,1,3,4,102324,5,7,126.0
4,0,2,11,2,1,2,3,100619,5,7,126.0


In [6]:
df.drop_duplicates(inplace=True)

In [7]:
io = InputOptimization()

In [8]:
io.generate_profile_report(df, '../01_EDA/reports/', 'Duplicates Removed')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 298.51it/s]


y = m1x1 + m2x2 + ....

In [9]:
df.head()

,Gender,Generation,Term Sub Reason,Employee Grade,Location,Career Bucket,Age Bucket,Manager ID,Talent,Department,active_months
0,1,2,1,1,1,1,2,102292,5,2,59.0
1,1,2,1,2,1,2,2,100411,5,3,4.0
2,0,2,1,1,1,1,2,102304,5,5,26.0
3,0,3,1,3,1,3,4,102324,5,7,126.0
4,0,2,11,2,1,2,3,100619,5,7,126.0


In [10]:
df.corr()

,Gender,Generation,Term Sub Reason,Employee Grade,Location,Career Bucket,Age Bucket,Manager ID,Talent,Department,active_months
Gender,1.000000,-0.150472,0.035157,-0.201826,-0.003756,-0.210651,-0.225511,-0.114667,0.036896,0.014237,-0.153644
Generation,-0.150472,1.000000,-0.054108,0.549182,0.139866,0.630168,0.730308,0.228114,0.009750,-0.013189,0.392650
Term Sub Reason,0.035157,-0.054108,1.000000,0.008616,0.040181,0.070083,0.001541,-0.070333,-0.064148,0.157438,-0.060258
Employee Grade,-0.201826,0.549182,0.008616,1.000000,0.149896,0.615055,0.666823,0.256370,-0.067168,-0.006530,0.330220
Location,-0.003756,0.139866,0.040181,0.149896,1.000000,0.107508,0.158126,0.211064,0.038083,0.091421,0.103257
Career Bucket,-0.210651,0.630168,0.070083,0.615055,0.107508,1.000000,0.798096,0.191164,-0.108371,0.145091,0.392734
Age Bucket,-0.225511,0.730308,0.001541,0.666823,0.158126,0.798096,1.000000,0.240877,-0.016604,0.036400,0.468850
Manager ID,-0.114667,0.228114,-0.070333,0.256370,0.211064,0.191164,0.240877,1.000000,-0.029507,0.286674,0.208727
Talent,0.036896,0.009750,-0.064148,-0.067168,0.038083,-0.108371,-0.016604,-0.029507,1.000000,-0.062697,0.012793
Department,0.014237,-0.013189,0.157438,-0.006530,0.091421,0.145091,0.036400,0.286674,-0.062697,1.000000,0.048544


In [ ]:
fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot()
plt.show()

In [12]:
df['Generation'].corr(df['Employee Grade']) *100

np.float64(54.91815479935123)

In [13]:
df.columns[:]

Index(['Gender', 'Generation', 'Term Sub Reason', 'Employee Grade', 'Location',
       'Career Bucket', 'Age Bucket', 'Manager ID', 'Talent', 'Department',
       'active_months'],
      dtype='object')

In [14]:
def correlation_table(df:pd.DataFrame):
    correlations = []
    count = 1
    for column1 in df.columns:
        for column2 in df.columns[count:]:
            corr = df[column1].corr(df[column2])
            correlations.append([column1, column2, corr])

        count = count + 1

    df_correlation = pd.DataFrame(correlations)
    df_correlation.columns = ['Feature 1', 'Feature 2', 'Correlation']
    df_correlation = df_correlation.sort_values(by='Correlation', ascending=False)
    return df_correlation

In [15]:
correlation_table(df).head()

,Feature 1,Feature 2,Correlation
40,Career Bucket,Age Bucket,0.798096
14,Generation,Age Bucket,0.730308
29,Employee Grade,Age Bucket,0.666823
13,Generation,Career Bucket,0.630168
28,Employee Grade,Career Bucket,0.615055


Regression Function

In [16]:
list(range(1, 11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [17]:
def linear_regression(df:pd.DataFrame, iteration=1):

    result = []
    for num in range(1, iteration+1):

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        scaler = StandardScaler()
        XS_train = scaler.fit_transform(X_train)
        XS_test = scaler.transform(X_test)

        model = LinearRegression()
        model.fit(XS_train, y_train)

        # Testing Model
        y_pred = model.predict(XS_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = math.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = model.score(XS_test, y_test) * 100

        result.append([model, mse, rmse, mae, r2])
    
    df_result = pd.DataFrame(result)
    df_result.columns = ['Model', 'MSE', 'RMSE', 'MAE', 'R2']
    df_result = df_result.sort_values('R2', ascending=False)

    return df_result
        

In [18]:
df.drop(['Career Bucket', 'Generation', 'Employee Grade'], axis=1, inplace=True)

In [19]:
correlation_table(df).head(5)

,Feature 1,Feature 2,Correlation
21,Age Bucket,active_months,0.468850
23,Manager ID,Department,0.286674
18,Age Bucket,Manager ID,0.240877
14,Location,Manager ID,0.211064
24,Manager ID,active_months,0.208727


In [20]:
lm_models = linear_regression(df, 100)
lm_models

,Model,MSE,RMSE,MAE,R2
58,LinearRegression(),811.537036,28.487489,22.887741,30.569251
26,LinearRegression(),943.650073,30.718888,24.610801,29.737324
96,LinearRegression(),966.060127,31.081508,24.931480,28.870166
69,LinearRegression(),1021.013799,31.953307,25.443897,28.188033
68,LinearRegression(),820.846728,28.650423,23.167960,28.099367
...,...,...,...,...,...
47,LinearRegression(),1073.056261,32.757537,26.235823,7.197523
9,LinearRegression(),819.053300,28.619107,22.918307,6.384249
57,LinearRegression(),1080.310293,32.868074,26.382218,5.588860
80,LinearRegression(),1102.070815,33.197452,25.852363,4.016905


In [21]:
lm_models.loc[94]['Model']

LinearRegression()

In [22]:
df.head()

,Gender,Term Sub Reason,Location,Age Bucket,Manager ID,Talent,Department,active_months
0,1,1,1,2,102292,5,2,59.0
1,1,1,1,2,100411,5,3,4.0
2,0,1,1,2,102304,5,5,26.0
3,0,1,1,4,102324,5,7,126.0
4,0,11,1,3,100619,5,7,126.0


In [24]:
df.to_csv('../00_Data/06_CorrelationDrop.csv', index=False)